In [3]:
import json
from enum import Enum
from collections import namedtuple
from ipynb.fs.full.Critical_Section import *

In [4]:
class CP_clock:
    clock = []
    
class lock_event:
    lock_clock = []

### Read the events from log

In [8]:
events, critical_section_list = gen_event_cs("temp.log")
for x in critical_section_list:
    print(x)

Thread : 1 
Locks : set() 
Read Objects : set() 
Write Objects : {'phi_x'} 
Start : 0 
End : 0 
Child Sections :

